# Monthly_Yearly_Rainfall_DailyAvg_&_Sum_Lat_Long
#### 1. Monthly Rainfall Daily_Avg & Sum Lat Long Wise
#### 2. Yearly Rainfall Daily_Avg & Sum Lat Long Wise

### 1. Monthly Rainfall Daily_Avg & Sum Lat Long Wise

### Output Table
Latitude | Longitude | Sum_1 | Mean_1 | Sum_2 | Mean_2 | ... | Sum_12 | Mean_12


In [2]:
import pandas as pd
import os
import calendar

# Function to process each CSV file and calculate monthly sum and average daily rainfall
def process_rainfall_data(file_path, output_file, year):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Inspect the columns
    print(f"Columns in the CSV: {df.columns.tolist()}")  # Check column names
    
    # Clean up column names by stripping whitespace
    df.columns = df.columns.str.strip()
    
    # Check if latitude and longitude columns are present
    if 'latitude' not in df.columns or 'longitude' not in df.columns:
        raise KeyError("The required columns 'latitude' and 'longitude' are missing from the CSV file.")
    
    # Filter rows for the specified year
    df = df[df['time'].str.startswith(year)]
    
    # Convert 'time' column to datetime format
    df['time'] = pd.to_datetime(df['time'], errors='coerce')
    
    # Drop rows with invalid timestamps
    df = df.dropna(subset=['time'])
    
    # Extract year and month from the 'time' column
    df['year'] = df['time'].dt.year
    df['month'] = df['time'].dt.month
    
    # Group by latitude, longitude, and month to calculate the sum
    monthly_stats = df.groupby(['year', 'month', 'longitude', 'latitude']).agg(
        monthly_sum=('APCP_sfc', 'sum')
    ).reset_index()
    
    # Calculate the number of days in each month
    monthly_stats['days_in_month'] = monthly_stats['month'].apply(lambda m: calendar.monthrange(int(year), m)[1])
    
    # Calculate average daily rainfall by dividing total monthly rainfall by the number of days
    monthly_stats['avg_daily_rainfall'] = monthly_stats['monthly_sum'] / monthly_stats['days_in_month']
    
    # Pivot the data to have separate columns for each month's sum and daily average
    pivot_sum = monthly_stats.pivot_table(index=['latitude', 'longitude'], 
                                          columns='month', 
                                          values='monthly_sum', 
                                          fill_value=0).add_prefix('Sum_')
    
    pivot_avg_daily = monthly_stats.pivot_table(index=['latitude', 'longitude'], 
                                                columns='month', 
                                                values='avg_daily_rainfall', 
                                                fill_value=0).add_prefix('Mean_')
    
    # Combine the sum and average daily pivot tables
    combined = pd.concat([pivot_sum, pivot_avg_daily], axis=1)
    
    # Reorder columns for the desired layout: sum_1, mean_1, sum_2, mean_2, ..., sum_12, mean_12
    new_columns = []
    for month in range(1, 13):
        new_columns.append(f'Sum_{month}')
        new_columns.append(f'Mean_{month}')
    
    # Sort and reorder columns
    combined = combined[new_columns]
    
    # Reset index to get 'latitude' and 'longitude' as columns
    combined = combined.reset_index()

    # Export the result to a new CSV file
    combined.to_csv(output_file, index=False)
    print(f"Processed and saved: {output_file}")

# Function to iterate through all files and process them
def process_all_files(input_folder, output_folder):
    for file_name in os.listdir(input_folder):
        if file_name.startswith('rain') and file_name.endswith('.csv'):
            year = file_name[4:8]  # Extract year from filename (e.g., '2012' from 'rain2012.csv')
            input_file = os.path.join(input_folder, file_name)
            output_file = os.path.join(output_folder, f"monthly_rain{year}.csv")
            process_rainfall_data(input_file, output_file, year)

# Define input and output folders
input_folder = r"Z:\Rainfall data and processing\Rainfall_data"  # Replace with the path to your input folder
output_folder = r"Z:\Rainfall data and processing\Monthly_Rainfall_Avg_Sum_Lat_Long"  # Replace with the path to your output folder

# Process all CSV files
process_all_files(input_folder, output_folder)


Columns in the CSV: ['time', 'longitude', 'latitude', 'APCP_sfc']
Processed and saved: Z:\Rainfall data and processing\Monthly_Rainfall_Avg_Sum_Lat_Long\monthly_rain2012.csv
Columns in the CSV: ['time', 'longitude', 'latitude', 'APCP_sfc']
Processed and saved: Z:\Rainfall data and processing\Monthly_Rainfall_Avg_Sum_Lat_Long\monthly_rain2013.csv
Columns in the CSV: ['time', 'longitude', 'latitude', 'APCP_sfc']
Processed and saved: Z:\Rainfall data and processing\Monthly_Rainfall_Avg_Sum_Lat_Long\monthly_rain2014.csv
Columns in the CSV: ['time', 'longitude', 'latitude', 'APCP_sfc']
Processed and saved: Z:\Rainfall data and processing\Monthly_Rainfall_Avg_Sum_Lat_Long\monthly_rain2015.csv
Columns in the CSV: ['time', 'longitude', 'latitude', 'APCP_sfc']
Processed and saved: Z:\Rainfall data and processing\Monthly_Rainfall_Avg_Sum_Lat_Long\monthly_rain2016.csv
Columns in the CSV: ['time', 'longitude', 'latitude', 'APCP_sfc']
Processed and saved: Z:\Rainfall data and processing\Monthly_Rai

### 2. Yearly Rainfall Daily Avg & Sum Lat Long Wise

### Output Table
Latitude | Longitude | Year Sum | Year Mean


In [6]:
import pandas as pd
import os
import calendar

# Function to process each CSV file and calculate yearly sum and average
def process_rainfall_data(file_path, output_file, year):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Filter rows for the specified year
    df = df[df['time'].str.startswith(year)]
    
    # Convert 'time' column to datetime format
    df['time'] = pd.to_datetime(df['time'], errors='coerce')
    
    # Drop rows with invalid timestamps
    df = df.dropna(subset=['time'])
    
    # Extract year and month from the 'time' column
    df['year'] = df['time'].dt.year
    df['month'] = df['time'].dt.month
    
    # Group by latitude and longitude to calculate yearly sum and mean
    yearly_stats = df.groupby(['year', 'longitude', 'latitude']).agg(
        yearly_sum=('APCP_sfc', 'sum'),
    ).reset_index()
    
    # Calculate the number of days in the year
    yearly_stats['days_in_year'] = yearly_stats['year'].apply(lambda x: 366 if calendar.isleap(x) else 365)
    
    # Calculate average daily rainfall by dividing total yearly rainfall by the number of days
    yearly_stats['yearly_mean'] = yearly_stats['yearly_sum'] / yearly_stats['days_in_year']
    
    # Keep only the relevant columns (latitude, longitude, Year Sum, Year Mean)
    yearly_result = yearly_stats[['latitude', 'longitude', 'yearly_sum', 'yearly_mean']].reset_index(drop=True)

    # Rename columns for clarity
    yearly_result.columns = ['latitude', 'longitude', 'Year Sum', 'Year Mean']

    # Export the result to a new CSV file
    yearly_result.to_csv(output_file, index=False)
    print(f"Processed and saved: {output_file}")

# Function to iterate through all files and process them
def process_all_files(input_folder, output_folder):
    for file_name in os.listdir(input_folder):
        if file_name.startswith('rain') and file_name.endswith('.csv'):
            year = file_name[4:8]  # Extract year from filename (e.g., '2012' from 'rain2012.csv')
            input_file = os.path.join(input_folder, file_name)
            output_file = os.path.join(output_folder, f"yearly_rain{year}.csv")
            process_rainfall_data(input_file, output_file, year)

# Define input and output folders
input_folder = r"Z:\Rainfall data and processing\Rainfall_data"  # Replace with the path to your input folder
output_folder = r"Z:\Rainfall data and processing\Yearly_Rainfall_Avg_Sum_Lat_Long"  # Replace with the path to your output folder

# Process all CSV files
process_all_files(input_folder, output_folder)


Processed and saved: Z:\Rainfall data and processing\Yearly_Rainfall_Avg_Sum_Lat_Long\yearly_rain2012.csv
Processed and saved: Z:\Rainfall data and processing\Yearly_Rainfall_Avg_Sum_Lat_Long\yearly_rain2013.csv
Processed and saved: Z:\Rainfall data and processing\Yearly_Rainfall_Avg_Sum_Lat_Long\yearly_rain2014.csv
Processed and saved: Z:\Rainfall data and processing\Yearly_Rainfall_Avg_Sum_Lat_Long\yearly_rain2015.csv
Processed and saved: Z:\Rainfall data and processing\Yearly_Rainfall_Avg_Sum_Lat_Long\yearly_rain2016.csv
Processed and saved: Z:\Rainfall data and processing\Yearly_Rainfall_Avg_Sum_Lat_Long\yearly_rain2017.csv
Processed and saved: Z:\Rainfall data and processing\Yearly_Rainfall_Avg_Sum_Lat_Long\yearly_rain2018.csv
Processed and saved: Z:\Rainfall data and processing\Yearly_Rainfall_Avg_Sum_Lat_Long\yearly_rain2019.csv
Processed and saved: Z:\Rainfall data and processing\Yearly_Rainfall_Avg_Sum_Lat_Long\yearly_rain2020.csv
Processed and saved: Z:\Rainfall data and proc